## 노트북 내용

이 노트북은 watsonx.ai를 사용한 RAG (Retrieval Augmented Generation)을 테스트 하기 위한 예제입니다. 여기에는 data retrieval, 지식 기본 정보에서 유사도검색 및 모델이 생성한 결과를 확인하는 내용이 포함되어 있습니다.
이를 잘 이해하기 위해서는 Python에 대한 기본 지식이 필요하며 모든 코드는 Python 3.12 으로 작성되어 있습니다.

### RAG (Retrieval Augmented Generation) 
Retrieval Augmented Generation (RAG)는 자연어로 지식 기반 데이터베이스에 질문이나 사실적인 정보를 사용하기를 원하는 다양한 usecase에 활용가능한 패턴입니다.

이 예제는 RAG의 가장 단순한 형태로서 세 가지 단계로 구성되어 있습니다. 

- 문답으로 된 엑셀데이터를 문답으로 나누고 embedding하여 지식 기반 데이터베이스 구축
- 지식 기반 데이터베이스로부터 사용자 질문과 가장 유사한 답을 추출
- 문답의 내용을 토큰화

## 내용

이 노트북은 다음과 같은 단계로 구성되어 있습니다ㅏ.

- [환경설정](#setup)
- [chroma 데이터베이스 컬렉션 생성](#create_collection)
- [Excel 파일 내용 읽고 출력](#read_excel)
- [Excel 파일을 임베딩하여 데이터베이스 구축](#build_base)
- [사용자 질문에 대한 답변 생성](#predict)

<a id="setup"></a>
## 환경 설정

이 노트북에 있는 샘플 코드를 실행하기 전에 다음 작업을 완료해야 합니다.

-  필요한 패키지를 설치해야 한다

    *chromadb: chromadb 데이터베이스에 데이터 삽입, 조회, 업데이트 및 삭제

    *pandas: 데이터 프레임 생성, 데이터 분석 및 시각화

    *sentence_transformer: 문장 임베딩 생성 및 문장 유사도 계산

    *xlrds: Excel 파일에서 데이터 읽기 및 처리

    *tqdm: 반복 작업 진행률 표시

In [ ]:
!pip install "chromadb==0.5.3"
!pip install "xlrd==2.0.1"
!pip install "pandas==2.1.4"
!pip install "sentence-transformers==3.0.1"
!pip install "tqdm==4.66.4"
!pip install "torch==2.3.1"

<a id="create_collection"></a>
## ChromaDB에 새로운 컬렉션 생성

In [ ]:
import chromadb
client = chromadb.PersistentClient()

try: 
    if client.get_collection(name="answers") is not None:
        client.delete_collection(name="answers")
except:
   pass

answers = client.create_collection(
        name="answers"
)

<a id="read_excel"></a>
## pandas 라이브러리를 사용하여 엑셀 파일을 읽고 출력

In [ ]:
import pandas as pd
import io
import os

filename = os.path.join(os.getcwd(), 'data', 'ChatbotData.xls')
df = pd.read_excel(filename)
print(df.head())

<a id="build_base"></a>
## Excel 파일을 임베딩하여 데이터베이스 구축

pandas DataFrame에서 데이터를 읽어 문장 임베딩을 생성하고 이를 chunk 단위로 처리하여 answers라는 객체에 추가

In [ ]:
import tqdm
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

ids = []
metadatas = []
embeddings = []

for row in tqdm.tqdm(df.iterrows()):
    index = row[0]
    query = row[1].Q
    answer = row[1].A
    
    metadata = {
        "query": query,
        "answer": answer
    }
    
    embedding = model.encode(query, normalize_embeddings=True)
    
    ids.append(str(index))
    metadatas.append(metadata)
    embeddings.append(embedding)
    
chunk_size = 256  # 한 번에 처리할 chunk 크기 설정
total_chunks = len(ids)

embeddings = [ e.tolist() for e in tqdm.tqdm(embeddings)]

for chunk_idx in tqdm.tqdm(range(total_chunks)):
    
    # chunk 단위로 데이터 자르기
    chunk_embeddings = embeddings[chunk_idx]
    chunk_ids = ids[chunk_idx]
    chunk_metadatas = metadatas[chunk_idx]
    
    # chunk를 answers에 추가
    answers.add(embeddings=chunk_embeddings, ids=chunk_ids, metadatas=chunk_metadatas)

<a id="predict"></a>
## 사용자 질문에 대한 답변 생성

In [ ]:
embeddings = model.encode("어제 여자친구랑 헤어졌다", normalize_embeddings=True)
result = answers.query(
    query_embeddings=embeddings.tolist(),
    n_results=1
)
print(result)

Hugging Face의 transformers 라이브러리를 사용하여 텍스트를 토큰화

In [ ]:
from transformers import AutoTokenizer
# 사전 학습된 모델 이름
MODEL = 'sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens'
tokenizer = AutoTokenizer.from_pretrained(MODEL)
result = tokenizer(df.loc[3, 'Q'], add_special_tokens = True, truncation = True, padding = "max_length", return_attention_mask = True, return_tensors = "pt")
print(result)

In [ ]:
tokens = tokenizer.convert_ids_to_tokens(result.input_ids[0])
print(tokens)

In [ ]:
print(tokenizer.convert_tokens_to_string(tokens))

In [ ]:
print(len(embeddings))
print(embeddings)